In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#References: 

#@misc{NBeatsPRemy,
#  author = {Philippe Remy},
#  title = {N-BEATS: Neural basis expansion analysis for interpretable time series forecasting},
#  year = {2020},
#  publisher = {GitHub},
#  journal = {GitHub repository},
#  howpublished = {\url{https://github.com/philipperemy/n-beats}},
#}



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore', message='Setting attributes')

#!pip install nbeats-keras  # Install nbeats-kearas
from nbeats_keras.model import NBeatsNet as NBeatsKeras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Sunspot data
data will hold series of Monthly Mean Total Sunspot Number

In [ ]:
df = pd.read_csv('../input/sunspots/Sunspots.csv')
data = pd.DataFrame()
data['value'] = df['Monthly Mean Total Sunspot Number']

## Initialize NBeats
num_samples : is total values in dataframe: 3265

time_steps  : is a time lags 

input_dim:  1 forecast at a time

output_dim: 1 forecast at a time

In [ ]:
num_samples, time_steps, input_dim, output_dim = data.shape[0], 5, 1, 1

backend = NBeatsKeras(
            backcast_length=time_steps, forecast_length=output_dim,
            stack_types=(NBeatsKeras.GENERIC_BLOCK, NBeatsKeras.GENERIC_BLOCK),
            nb_blocks_per_stack=2, thetas_dim=(4, 4), share_weights_in_stack=True,
            hidden_layer_units=64
            )
backend.compile(loss='mae', optimizer='adam')

## Prepare data 
x is converted to 3 dimentional (total samples/lags,lags,input_dimentions)

y is a mean of 3 lags in x values

Ex: if data is of total lenght 3 as below and lag value is also 3

data =  [3,6,3]

x = [3,6,3]

y = 4



In [ ]:
# Definition of the data. The problem to solve is to find f such as | f(x) - y | -> 0.
        # where f = np.mean.
#x = np.random.uniform(size=(num_samples, time_steps, input_dim))
#y = np.mean(x, axis=1, keepdims=True)

x = data.values.reshape(num_samples//5, time_steps, input_dim)
y = np.mean(x, axis=1, keepdims=True)

# Split data into training and testing datasets.
c = num_samples // 10
x_train, y_train, x_test, y_test = x[c:], y[c:], x[:c], y[:c]
test_size = len(x_test)

## Model Taining 

All are default settings as in initial repo by

@misc{NBeatsPRemy,
  author = {Philippe Remy},
  title = {N-BEATS: Neural basis expansion analysis for interpretable time series forecasting},
  year = {2020},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/philipperemy/n-beats}},
}

In [ ]:
# Train the model.
print('Training...')
backend.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=128)

In [ ]:
# Save the model for later.
backend.save('n_beats_model.h5')

In [ ]:
# Predict on the testing set (forecast).
predictions_forecast = backend.predict(x_test)
np.testing.assert_equal(predictions_forecast.shape, (test_size, backend.forecast_length, output_dim))

## Plot values for initial inspection

In [ ]:
plt.plot(predictions_forecast.reshape(326))
plt.plot(y_test.reshape(326))

## Find MAP and MSE

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

print(mean_absolute_error(predictions_forecast.reshape(326),y_test.reshape(326)))
print(mean_squared_error(predictions_forecast.reshape(326),y_test.reshape(326)))

In [ ]:
 # Predict on the testing set (backcast).
predictions_backcast = backend.predict(x_test, return_backcast=True)
np.testing.assert_equal(predictions_backcast.shape, (test_size, backend.backcast_length, output_dim))

In [ ]:
plt.plot(predictions_backcast.reshape(1630))
plt.plot(y_train.reshape(327))
plt.plot(data['value'].values)

In [ ]:
# Load the model.
model_2 = NBeatsKeras.load('n_beats_model.h5')

In [ ]:
np.testing.assert_almost_equal(predictions_forecast, model_2.predict(x_test))